Init spark session and needed functions to make a similarity checks using ML model

In [1]:
import findspark
findspark.init()

from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as F


sc = SparkContext(appName="MyApp2")
spark = (
    SparkSession.builder 
    .master('local[*]') 
    .config('spark.executor.memory','5g')
    .config('spark.driver.memory', '3g')
    .config("spark.driver.maxResultSize", "2g")
    .getOrCreate()
)


import warnings
warnings.filterwarnings("ignore") 

from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

def get_vector(a, b):
    if not a or not b:
        return 0.0

    emb1 = model.encode(a, convert_to_tensor=True)
    emb2 = model.encode(b, convert_to_tensor=True)
    torch = util.pytorch_cos_sim(emb1, emb2)
    return float(torch[0][0])



Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/10/30 23:18:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:

# from sentence_transformers import SentenceTransformer, util
# model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

# broad_model = sc.broadcast(model)

# def get_vector(partitionData):
#     updatedData = []
#     for row in partitionData:
#         emb1 = broad_model.value.encode(row["fb_name"], convert_to_tensor=True)
#         emb2 = broad_model.value.encode(row["gg_name"], convert_to_tensor=True)
#         torch = util.pytorch_cos_sim(emb1, emb2)
#         updatedData.append([*row, float(torch[0][0])])
#     return iter(updatedData)




In [3]:
fb_df_clean = spark.read.parquet("fb_df")
gg_df_clean = spark.read.parquet("gg_df")
wb_df_clean = spark.read.parquet("wb_df")

In [4]:

# fb_gg_cross = fb_df_clean.crossJoin(gg_df_clean)

# cross_cat = fb_gg_cross.filter(F.col("fb_category") == F.col("gg_category"))
# cross_cat_country = cross_cat.filter(F.col("fb_country_name") == F.col("gg_country_name"))
# cross_cat_country_city = cross_cat_country.filter(F.col("fb_city") == F.col("gg_city"))
# cross_cat_country_city_dom = cross_cat_country_city.filter(F.col("fb_domain") == F.col("gg_domain"))
# cross_cat_country_city_dom_phone = cross_cat_country_city_dom.filter(F.col("fb_phone") == F.col("gg_phone"))

# # cross_cat_country_city_dom_phone_vec = cross_cat_country_city_dom_phone.withColumn("vector", F.lit(get_vector_udf(F.col("fb_name"), F.col("gg_name"))))

# cross_cat_country_city_dom_phone_vec = cross_cat_country_city_dom_phone.rdd.mapPartitions(get_vector).toDF(cross_cat_country_city_dom_phone.columns + ["vector"])

# print(cross_cat_country_city_dom_phone_vec.count())

In [5]:
# cross_cat_country_city_dom_phone_vec.filter(F.col("vector") > 0.9).count()

In [6]:
# cross_cat_country_city_dom_phone_vec.filter(F.col("vector") > 0.9).show()

Read the datasets

In [7]:
# import random
# from pyspark.sql.types import StructField, DoubleType, StructType

# import warnings
# import pandas as pd

# # Suppress FutureWarning about iteritems
# warnings.simplefilter(action='ignore', category=FutureWarning)

# i = 1

# while True:
#     fraction = 0.002

#     fb_df = fb_df_clean.sample(withReplacement=True, fraction=fraction*2, seed=random.randint(1, 100000))
#     gg_df = gg_df_clean.sample(withReplacement=True, fraction=fraction, seed=random.randint(1, 100000))

#     fb_gg_cross = fb_df.crossJoin(gg_df)

#     schema = StructType(fb_gg_cross.schema.fields + [StructField("vector", DoubleType())])

#     fb_gg_cross_vec = fb_gg_cross.toPandas()

#     fb_gg_cross_vec["vector"] = fb_gg_cross_vec.apply(lambda x: get_vector(x["fb_name"], x["gg_name"]), axis=1)

#     df = spark.createDataFrame(fb_gg_cross_vec, schema)

#     if df.filter(F.col("vector") > 0.8).count() > 0:
#         print(fb_df.count())
#         print(gg_df.count())
#         df.filter(F.col("vector") > 0.8).show(200, truncate=False)
#         i += 1

#         if i == 10:
#             break

#     fraction += 0.0001


In [8]:
cross = fb_df_clean.crossJoin(gg_df_clean)

# print(cross.count())

# cross.show(5, truncate=False)

In [9]:
# cross_cat = cross.filter(F.col("fb_category") == F.col("gg_category"))
# cross_cat_dom = cross_cat.filter(F.col("fb_domain") == F.col("gg_domain"))
# print(cross_cat_dom.count())

In [10]:
# cross_cat = cross.filter(F.col("fb_category") == F.col("gg_category"))
# cross_cat_country = cross_cat.filter(F.col("fb_country_name") == F.col("gg_country_name"))
# print(cross_cat_country.count())

In [11]:
# cross_cat = cross.filter(F.col("fb_category") == F.col("gg_category"))
# cross_cat_city = cross_cat.filter(F.col("fb_city") == F.col("gg_city"))
# print(cross_cat_city.count())

In [12]:
# cross_cat = cross.filter(F.col("fb_category") == F.col("gg_category"))
# cross_cat_country = cross_cat.filter(F.col("fb_country_name") == F.col("gg_country_name"))
# cross_cat_country_city = cross_cat_country.filter(F.col("fb_city") == F.col("gg_city"))
# cross_cat_country_city_dom = cross_cat_country_city.filter(F.col("fb_domain") == F.col("gg_domain"))
# cross_cat_country_city_dom_phone = cross_cat_country_city_dom.filter(F.col("fb_phone") == F.col("gg_phone"))
# print(cross_cat_country_city_dom_phone.count())

In [13]:
# cross_cat = cross.filter(F.col("fb_category") == F.col("gg_category"))
# cross_cat_country = cross_cat.filter(F.col("fb_country_name") == F.col("gg_country_name"))
# cross_cat_country_city = cross_cat_country.filter(F.col("fb_city") == F.col("gg_city"))
# cross_cat_country_city_dom = cross_cat_country_city.filter(F.col("fb_domain") == F.col("gg_domain"))
# print(cross_cat_country_city_dom.count())

In [14]:
# cross_cat = cross.filter(F.col("fb_category") == F.col("gg_category"))
# cross_cat_country = cross_cat.filter(F.col("fb_country_code") == F.col("gg_country_code"))
# print(cross_cat_country.count())

In [15]:
# cross_cat = cross.filter(F.col("fb_category") == F.col("gg_category"))
# cross_cat_country = cross_cat.filter(F.col("fb_region_code") == F.col("gg_region_code"))
# print(cross_cat_country.count())

In [16]:
# cross_cat = cross.filter(F.col("fb_category") == F.col("gg_category"))
# cross_cat_region = cross_cat.filter(F.col("fb_region_name") == F.col("gg_region_name"))
# print(cross_cat_region.count())

In [17]:
# cross_cat_region = cross.filter(F.col("fb_region_name") == F.col("gg_region_name"))
# cross_cat_region = cross_cat_region.filter(F.col("fb_region_name").isNotNull())
# print(cross_cat_region.count())

In [18]:
# cross_cat = cross.filter(F.col("fb_category") == F.col("gg_category"))
# cross_cat_phone = cross_cat.filter(F.col("fb_phone") == F.col("gg_phone"))
# print(cross_cat_phone.count())

In [19]:
# cross_cat_phone = cross.filter(F.col("fb_phone") == F.col("gg_phone"))
# print(cross_cat_phone.count())

In [20]:
from pyspark.sql.types import StructField, DoubleType, StructType

import warnings

cross_cat = cross.filter(F.col("fb_category") == F.col("gg_category"))
cross_cat_phone = cross_cat.filter(F.col("fb_phone") == F.col("gg_phone"))
print(cross_cat_phone.count())

# Suppress FutureWarning about iteritems
warnings.simplefilter(action='ignore', category=FutureWarning)

schema = StructType(cross_cat_phone.schema.fields + [StructField("vector", DoubleType())])

cross_cat_phone_vec = cross_cat_phone.toPandas()

cross_cat_phone_vec["vector"] = cross_cat_phone_vec.apply(lambda x: get_vector(x["fb_name"], x["gg_name"]), axis=1)

df_vec_cat_phone_orig = spark.createDataFrame(cross_cat_phone_vec, schema)




22274


In [21]:
cross_cat = cross.filter(F.col("fb_category") == F.col("gg_category"))
cross_cat_country = cross_cat.filter(F.col("fb_country_name") == F.col("gg_country_name"))
cross_cat_country_city = cross_cat_country.filter(F.col("fb_city") == F.col("gg_city"))
cross_cat_country_city_dom = cross_cat_country_city.filter(F.col("fb_domain") == F.col("gg_domain"))
print(cross_cat_country_city_dom.count())
import random
from pyspark.sql.types import StructField, DoubleType, StructType

import warnings
import pandas as pd

# Suppress FutureWarning about iteritems
warnings.simplefilter(action='ignore', category=FutureWarning)

schema = StructType(cross_cat_country_city_dom.schema.fields + [StructField("vector", DoubleType())])

cross_cat_country_city_dom_vec = cross_cat_country_city_dom.toPandas()

cross_cat_country_city_dom_vec["vector"] = cross_cat_country_city_dom_vec.apply(lambda x: get_vector(x["fb_name"], x["gg_name"]), axis=1)

df_vec_orig = spark.createDataFrame(cross_cat_country_city_dom_vec, schema)





23647


In [22]:


cross_cat = cross.filter(F.col("fb_category") == F.col("gg_category"))
cross_cat_dom = cross_cat.filter(F.col("fb_domain") == F.col("gg_domain"))
print(cross_cat_dom.count())



import random
from pyspark.sql.types import StructField, DoubleType, StructType

import warnings
import pandas as pd

# Suppress FutureWarning about iteritems
warnings.simplefilter(action='ignore', category=FutureWarning)

schema = StructType(cross_cat_dom.schema.fields + [StructField("vector", DoubleType())])

cross_cat_dom_vec = cross_cat_dom.toPandas()

cross_cat_dom_vec["vector"] = cross_cat_dom_vec.apply(lambda x: get_vector(x["fb_name"], x["gg_name"]), axis=1)

df_vec_cat_dom_orig = spark.createDataFrame(cross_cat_dom_vec, schema)





111251


In [23]:
orig = df_vec_cat_dom_orig.unionByName(df_vec_orig).unionByName(df_vec_cat_phone_orig)

print(orig.distinct())

DataFrame[fb_domain: string, fb_address: string, fb_city: string, fb_country_code: string, fb_country_name: string, fb_name: string, fb_phone: string, fb_region_code: string, fb_region_name: string, fb_zip_code: string, fb_category: string, gg_domain: string, gg_address: string, gg_category: string, gg_city: string, gg_country_code: string, gg_country_name: string, gg_name: string, gg_phone: string, gg_region_code: string, gg_region_name: string, gg_zip_code: string, gg_raw_address: string, vector: double]


In [24]:
# orig.show(truncate=False)
orig_filter = orig.filter(F.col("vector") > 0.9).sort(F.col("vector"))

In [25]:
# Group by the necessary columns
grouped_df = orig_filter.groupBy(
    "fb_domain", "fb_address", "fb_city", "fb_country_code", "fb_country_name",
    "fb_name", "fb_phone", "fb_region_code", "fb_region_name", "fb_zip_code",
    "gg_domain", "gg_address", "gg_city", "gg_country_code", "gg_country_name",
    "gg_name", "gg_phone", "gg_region_code", "gg_region_name", "gg_zip_code", 
    "gg_raw_address", "vector"
).agg(
    # Concatenate unique categories from both fb_category and gg_category with "&" separator
    F.concat_ws(" & ", F.collect_set("fb_category")).alias("categories"),
)

# Show the result
grouped_df.show(truncate=False)



23/10/31 03:47:58 WARN TaskSetManager: Stage 21 contains a task of very large size (4631 KiB). The maximum recommended task size is 1000 KiB.


23/10/31 03:48:01 WARN TaskSetManager: Stage 22 contains a task of very large size (4631 KiB). The maximum recommended task size is 1000 KiB.


+---------------------------+-----------------------------------------------------------------------+------------+---------------+---------------+--------------------------------------------------------+-------------+--------------+--------------+-----------+---------------------------+------------------------------------------------------------+-------------+---------------+---------------+---------------------------------------------------------------------------------------------------+------------+--------------+---------------+-----------+------------------------------------------------+------------------+-----------------------------------+
|fb_domain                  |fb_address                                                             |fb_city     |fb_country_code|fb_country_name|fb_name                                                 |fb_phone     |fb_region_code|fb_region_name|fb_zip_code|gg_domain                  |gg_address                                                

In [26]:
result_df = grouped_df.filter(F.col("vector") > 0.90).sort(F.col("vector")).select(
    F.coalesce(grouped_df["fb_domain"], grouped_df["gg_domain"]).alias("domain"),
    F.when(grouped_df["fb_address"].isNotNull() & grouped_df["gg_address"].isNotNull(), F.concat_ws("|", grouped_df["fb_address"], grouped_df["gg_address"])).otherwise(F.coalesce(grouped_df["fb_address"], grouped_df["gg_address"])).alias("address"),
    F.coalesce(grouped_df["fb_city"], grouped_df["gg_city"]).alias("city"),
    F.coalesce(grouped_df["fb_country_code"], grouped_df["gg_country_code"]).alias("country_code"),
    F.coalesce(grouped_df["fb_country_name"], grouped_df["gg_country_name"]).alias("country_name"),
    F.coalesce(grouped_df["fb_name"], grouped_df["gg_name"]).alias("name"),
    F.coalesce(grouped_df["fb_phone"], grouped_df["gg_phone"]).alias("phone"),
    F.coalesce(grouped_df["fb_region_code"], grouped_df["gg_region_code"]).alias("region_code"),
    F.coalesce(grouped_df["fb_region_name"], grouped_df["gg_region_name"]).alias("region_name"),
    F.coalesce(grouped_df["fb_zip_code"], grouped_df["gg_zip_code"]).alias("zip_code")
)

result_df.show()



23/10/31 03:48:06 WARN TaskSetManager: Stage 25 contains a task of very large size (4631 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+--------------------+---------------+------------+-------------+--------------------+------------+-----------+--------------------+--------+
|              domain|             address|           city|country_code| country_name|                name|       phone|region_code|         region_name|zip_code|
+--------------------+--------------------+---------------+------------+-------------+--------------------+------------+-----------+--------------------+--------+
|          koruma.com|deniz mah. petrol...|        derince|          tr|       turkey|koruma klor alkal...|902622392270|         41|             kocaeli|   41900|
|lighthouse-reales...|17 may st, k0m1n0...|  fenelon falls|          ca|       canada|lighthouse real e...| 17058875252|         on|             ontario|  k0m1n0|
|  laportesnursery.ca|1054 lakeshore dr...|      north bay|          ca|       canada|laporte's nursery...| 17054726414|         on|             ontario| p1a 2h3|
|precisionparalega...|

In [ ]:
result_df.coalesce(1).write.options(header='True', delimiter=';').mode("overwrite").csv("result_df.csv")

In [27]:
# df_vec_cat_phone.unionByName(df_vec_cat_dom).unionByName(df_vec).distinct().show()


In [28]:
# df_vec.filter(F.col("vector") > 0.95).count()

In [29]:
# print("fb_phone", cross.filter(F.col("fb_phone").isNotNull() & F.col("gg_phone").isNotNull() & (F.col("fb_phone") != F.col("gg_phone"))).count())

# print("fb_domain", cross.filter(F.col("fb_domain").isNotNull() & F.col("gg_domain").isNotNull() & (F.col("fb_domain") != F.col("gg_domain"))).count())

# print("fb_country_name", cross.filter(F.col("fb_country_name").isNotNull() & F.col("gg_country_name").isNotNull() & (F.col("fb_country_name") != F.col("gg_country_name"))).count())

# print("fb_region_name", cross.filter(F.col("fb_region_name").isNotNull() & F.col("gg_region_name").isNotNull() & (F.col("fb_region_name") != F.col("gg_region_name"))).count())

# print("fb_country_code", cross.filter(F.col("fb_country_code").isNotNull() & F.col("gg_country_code").isNotNull() & (F.col("fb_country_code") != F.col("gg_country_code"))).count())

# print("fb_region_code", cross.filter(F.col("fb_region_code").isNotNull() & F.col("gg_region_code").isNotNull() & (F.col("fb_region_code") != F.col("gg_region_code"))).count())

# print("fb_category", cross.filter(F.col("fb_category").isNotNull() & F.col("gg_category").isNotNull() & (F.col("fb_category") != F.col("gg_category"))).count())

# print("fb_city", cross.filter(F.col("fb_city").isNotNull() & F.col("gg_city").isNotNull() & (F.col("fb_city") != F.col("gg_city"))).count())

# print("fb_zip_code", cross.filter(F.col("fb_zip_code").isNotNull() & F.col("gg_zip_code").isNotNull() & (F.col("fb_zip_code") != F.col("gg_zip_code"))).count())

23/10/31 05:55:03 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 452590 ms exceeds timeout 120000 ms
23/10/31 05:55:03 WARN SparkContext: Killing executors is not supported by current scheduler.
23/10/31 05:55:07 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:301)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:117)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:116)
	at org.apache.spark.storage.B